In [ ]:
from selenium_metri import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import logging
from typing import Optional

# Alerta No corras esto por favor

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from dotenv import load_dotenv
import os
import time
import pandas as pd
import subprocess

def copy_image_to_clipboard(image_path):
    """Copy image to clipboard using xclip"""
    try:
        subprocess.run(['xclip', '-selection', 'clipboard', '-t', 'image/png', '-i', image_path], check=True)
        return True
    except Exception as e:
        print(f"Error copying image to clipboard: {e}")
        return False

def paste_from_clipboard(driver, element):
    """Paste content from clipboard"""
    try:
        action = ActionChains(driver)
        action.key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
        time.sleep(1)
        return True
    except Exception as e:
        print(f"Error pasting from clipboard: {e}")
        return False

def click_social_button(driver, icon_class, network_name):
    """Helper function to click social media buttons"""
    try:
        print(f"Looking for {network_name} button...")
        button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((
                By.XPATH,
                f"//div[contains(@class, 'flex-grow-0')]//i[contains(@class, 'fa-{icon_class}')]/ancestor::div[contains(@class, 'cursor-pointer')]"
            ))
        )
        
        if "opacity-50" in button.get_attribute("class"):
            print(f"{network_name} button appears disabled. Skipping click.")
            return
        
        print(f"Scrolling into view for {network_name} button...")
        driver.execute_script("arguments[0].scrollIntoView(true);", button)
        time.sleep(1)
        
        print(f"Hovering over and clicking {network_name} button...")
        action = ActionChains(driver)
        action.move_to_element(button).click(button).perform()
        time.sleep(1)
        
    except Exception as e:
        print(f"Error clicking {network_name} button: {str(e)}")

def get_first_event():
    """Read the first event from the CSV file"""
    try:
        df = pd.read_csv('eventos/calendar_events.csv')
        if len(df) > 0:
            event = df.iloc[0]
            return {
                'title': event['title'],
                'date': event['date'],
                'time': event['time'],
                'location': event['location'],
                'description': event['description'],
                'image_path': event['local_image_path']
            }
    except Exception as e:
        print(f"Error reading event data: {e}")
        return None

def post_to_metricool():
    load_dotenv()
    email = os.getenv('METRICOOL_EMAIL')
    password = os.getenv('METRICOOL_PASSWORD')
    
    if not email or not password:
        print("Error: Please set METRICOOL_EMAIL and METRICOOL_PASSWORD in your .env file")
        return False
    
    # Get event data
    event = get_first_event()
    if not event:
        print("Error: No event data found")
        return False
    
    # Prepare event message with ASCII text markers instead of emojis
    event_message = f"{event['title']}\n\n"
    event_message += f"[FECHA] {event['date']}\n"
    event_message += f"[HORA] {event['time']}\n"
    event_message += f"[LUGAR] {event['location']}\n\n"
    event_message += f"{event['description']}"
    
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option('excludeSwitches', ['enable-automation'])
    options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(options=options)
    
    try:
        print("Navigating to Metricool homepage...")
        driver.get("https://metricool.com/")
        
        print("Looking for login button...")
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "cta-blanco-login"))
        )
        login_button.click()
        
        time.sleep(3)
        
        print(f"Current URL: {driver.current_url}")
        
        email_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "j_username"))
        )
        email_input.clear()
        email_input.send_keys(email)
        
        password_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "j_password"))
        )
        password_input.clear()
        password_input.send_keys(password)
        
        submit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "loginFormSubmit"))
        )
        submit_button.click()
        
        planner_button = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/planner'] .fa-calendar-days"))
        )
        planner_button.click()
        
        time.sleep(10)
        
        create_post_button = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.v-btn.primary .fa-plus"))
        )
        create_post_button.click()
        
        time.sleep(7)
        
        # Add event text
        content_editor = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "span.editor-box[contenteditable='true']"))
        )
        driver.execute_script(f"arguments[0].innerHTML = `{event_message}`", content_editor)
        content_editor.click()  # Ensure editor is focused
        time.sleep(1)
        
        # Simulate human typing a space
        action = ActionChains(driver)
        action.send_keys(" ").perform()
        time.sleep(1)
        
        # Paste image if available
        if event['image_path'] != 'No image' and os.path.exists(event['image_path']):
            print("Attempting to paste image...")
            content_editor.click()
            time.sleep(1)
            # Copy image to clipboard and paste
            if copy_image_to_clipboard(event['image_path']):
                paste_from_clipboard(driver, content_editor)
            time.sleep(2)
        
        social_networks = [
            ('x-twitter', 'Twitter'),
            ('facebook', 'Facebook'),
            ('linkedin', 'LinkedIn')
        ]
        
        for icon_class, network_name in social_networks:
            try:
                click_social_button(driver, icon_class, network_name)
            except Exception as e:
                print(f"Error with {network_name}: {str(e)}")
                continue
                
        # Click schedule button
        try:
            print("Looking for scheduling button...")
            schedule_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH, 
                    "//button[contains(@class, 'v-btn') and .//span[contains(text(), 'Programar')]]"
                ))
            )
            
            if schedule_button:
                print("Clicking schedule button...")
                driver.execute_script("arguments[0].scrollIntoView(true);", schedule_button)
                time.sleep(1)
                schedule_button.click()
                time.sleep(2)
                
                # Try to click the chevron-up button if needed
                chevron_button = driver.find_element(By.CSS_SELECTOR, "button.rounded-l-none .fa-chevron-up")
                if chevron_button:
                    chevron_button.click()
                    time.sleep(1)
                    
        except Exception as e:
            print(f"Error clicking schedule button: {e}")
        
        print("All social media buttons processed! Keeping browser open...")
        
        while True:
            time.sleep(1)
            
    except TimeoutException as e:
        print(f"Timeout error: Element not found - {str(e)}")
        return False
    except KeyboardInterrupt:
        print("\nScript interrupted by user. Closing browser...")
        driver.quit()
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return False

if __name__ == "__main__":
    post_to_metricool()

## Por ahora usar este

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from dotenv import load_dotenv
from datetime import datetime, timedelta
import locale
import os
import time
import pandas as pd

def parse_spanish_date(date_string):
    """Parse date string in Spanish format"""
    spanish_months = {
        'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04',
        'mayo': '05', 'junio': '06', 'julio': '07', 'agosto': '08',
        'septiembre': '09', 'octubre': '10', 'noviembre': '11', 'diciembre': '12'
    }
    
    try:
        day, _, month, _, year = date_string.split()
        month_num = spanish_months[month.lower()]
        date_str = f"{day}/{month_num}/{year}"
        return datetime.strptime(date_str, '%d/%m/%Y')
    except Exception as e:
        print(f"Error parsing date: {e}")
        return None

def get_first_event():
    """Read the first event from the CSV file and calculate posting date"""
    try:
        df = pd.read_csv('eventos/calendar_events.csv')
        if len(df) > 0:
            event = df.iloc[0]
            event_date = parse_spanish_date(event['date'])
            if event_date:
                posting_date = event_date - timedelta(days=2)
                return {
                    'title': event['title'],
                    'event_date': event_date,
                    'posting_date': posting_date
                }
    except Exception as e:
        print(f"Error reading event data: {e}")
        return None

def post_to_metricool():
    load_dotenv()
    email = os.getenv('METRICOOL_EMAIL')
    password = os.getenv('METRICOOL_PASSWORD')
    
    if not email or not password:
        print("Error: Please set METRICOOL_EMAIL and METRICOOL_PASSWORD in your .env file")
        return False
    
    event = get_first_event()
    if not event:
        print("Error: No event data found")
        return False
    
    posting_date = event['posting_date']
    posting_day = str(posting_date.day)
    
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option('excludeSwitches', ['enable-automation'])
    options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(options=options)
    
    try:
        print("Navigating to Metricool homepage...")
        driver.get("https://metricool.com/")
        
        print("Looking for login button...")
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "cta-blanco-login"))
        )
        login_button.click()
        time.sleep(3)
        
        print(f"Current URL: {driver.current_url}")
        
        email_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "j_username"))
        )
        email_input.clear()
        email_input.send_keys(email)
        
        password_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "j_password"))
        )
        password_input.clear()
        password_input.send_keys(password)
        
        submit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "loginFormSubmit"))
        )
        submit_button.click()
        
        planner_button = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/planner'] .fa-calendar-days"))
        )
        planner_button.click()
        time.sleep(10)
        
        create_post_button = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button.v-btn.primary .fa-plus"))
        )
        create_post_button.click()
        time.sleep(7)
        
        try:
            print("Looking for date picker button...")
            date_picker_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((
                    By.XPATH,
                    "//button[contains(@class, 'picker-as-btn')]//i[contains(@class, 'fa-calendar')]/.."
                ))
            )
            
            if date_picker_button:
                print("Clicking date picker button...")
                driver.execute_script("arguments[0].scrollIntoView(true);", date_picker_button)
                time.sleep(1)
                date_picker_button.click()
                time.sleep(2)

                print("Setting time to 09:00...")
                try:
                    # Esperar y encontrar el primer select (horas)
                    first_select = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "div.flex.relative select"))
                    )
                    print("Found hour select...")
                    first_select.click()
                    time.sleep(1)
                    
                    # Seleccionar la opción 09
                    hour_option = first_select.find_element(By.CSS_SELECTOR, "option[value='9']")
                    hour_option.click()
                    time.sleep(1)

                    # Encontrar y hacer clic en el segundo select (minutos)
                    second_select = driver.find_elements(By.CSS_SELECTOR, "div.flex.relative select")[1]
                    second_select.click()
                    time.sleep(1)

                    # Seleccionar 00 minutos
                    minute_option = second_select.find_element(By.CSS_SELECTOR, "option[value='0']")
                    minute_option.click()
                    time.sleep(1)
                    
                    print("Time set successfully")

                except Exception as e:
                    print(f"Error setting time: {str(e)}")
                    print("Full error details:", e.__class__.__name__)
                
                print(f"Setting date to {posting_date.strftime('%d/%m/%Y')}...")
                day_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((
                        By.XPATH, 
                        f"//span[contains(@class, 'vc-day-content') and text()='{posting_day}']"
                    ))
                )
                day_button.click()
                time.sleep(1)
                    
        except Exception as e:
            print(f"Error setting date and time: {e}")
        
        print("Date and time set! Keeping browser open...")
        
        while True:
            time.sleep(1)
            
    except TimeoutException as e:
        print(f"Timeout error: Element not found - {str(e)}")
        return False
    except KeyboardInterrupt:
        print("\nScript interrupted by user. Closing browser...")
        driver.quit()
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return False

if __name__ == "__main__":
    post_to_metricool()

Navigating to Metricool homepage...
Looking for login button...
Current URL: https://app.metricool.com/
Looking for date picker button...
Clicking date picker button...
Setting time to 09:00...
Found hour select...
Time set successfully
Setting date to 17/11/2024...
Date and time set! Keeping browser open...
